In [1]:
from typing import Annotated

from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


tool = TavilySearchResults(max_results=2)
tools = [tool]
llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [2]:
config = {"configurable": {"thread_id": "1"}}
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "I'm learning LangGraph. "
                    "Could you do some research on it for me?"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

I'm learning LangGraph. Could you do some research on it for me?
================================== Ai Message ==================================

[{'text': "Certainly! I'd be happy to research LangGraph for you. To get the most up-to-date and comprehensive information, I'll use the Tavily search engine to look this up. Let me do that for you now.", 'type': 'text'}, {'id': 'toolu_01RmzuPobKTxCEVU2RcjYawZ', 'input': {'query': 'LangGraph: what is it, features, and use cases in AI development'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_01RmzuPobKTxCEVU2RcjYawZ)
 Call ID: toolu_01RmzuPobKTxCEVU2RcjYawZ
  Args:
    query: LangGraph: what is it, features, and use cases in AI development
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "What Is LangGraph and How to Use It

In [3]:
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "Ya that's helpful. Maybe I'll "
                    "build an autonomous agent with it!"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Ya that's helpful. Maybe I'll build an autonomous agent with it!
================================== Ai Message ==================================

[{'text': "That's an excellent idea! Building an autonomous agent with LangGraph is a great way to learn the technology and explore its capabilities. Let me provide some more specific information and suggestions to help you get started with this project.", 'type': 'text'}, {'id': 'toolu_01RTGyoWcyHPLWbzMjoZPWWW', 'input': {'query': 'Building autonomous agents with LangGraph tutorial or guide'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_01RTGyoWcyHPLWbzMjoZPWWW)
 Call ID: toolu_01RTGyoWcyHPLWbzMjoZPWWW
  Args:
    query: Building autonomous agents with LangGraph tutorial or guide
================================= Tool Message =================================
Name: tavily_search_results_json

[{"ti

In [6]:
to_replay = None
for state in graph.get_state_history(config):
    print("Num Messages: ", len(state.values["messages"]), "Next: ", state.next, state.config)
    print("-" * 80)
    if len(state.values["messages"]) == 6:
        # We are somewhat arbitrarily selecting a specific state based on the number of chat messages in the state.
        to_replay = state

Num Messages:  8 Next:  () {'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1effceef-de4f-6cde-8008-0a5cec98f69a'}}
--------------------------------------------------------------------------------
Num Messages:  7 Next:  ('chatbot',) {'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1effceef-731a-65a4-8007-de904ccb8e84'}}
--------------------------------------------------------------------------------
Num Messages:  6 Next:  ('tools',) {'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1effceef-58ff-6aac-8006-a0aa0d02619e'}}
--------------------------------------------------------------------------------
Num Messages:  5 Next:  ('chatbot',) {'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1effceef-27e4-6cec-8005-11ab2f4873ec'}}
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ('__start__',) {'configurable': {'thread_id': '1', 'chec

In [5]:
print(to_replay.next)
print(to_replay.config)

('tools',)
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1effceef-58ff-6aac-8006-a0aa0d02619e'}}


Notice that the checkpoint's config (to_replay.config) contains a checkpoint_id timestamp. Providing this checkpoint_id value tells LangGraph's checkpointer to load the state from that moment in time. Let's try it below:

In [ ]:
# The `checkpoint_id` in the `to_replay.config` corresponds to a state we've persisted to our checkpointer.
for event in graph.stream(None, to_replay.config, stream_mode="values"):
    if "messages" in event:
        event["messages"][-1].pretty_print()

Notice that the graph resumed execution from the **action** node. You can tell this is the case since the first value printed above is the response from our search engine tool.
Congratulations! You've now used time-travel checkpoint traversal in LangGraph. Being able to rewind and explore alternative paths opens up a world of possibilities for debugging, experimentation, and interactive applications.